In [0]:
!pip install names

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bsp
import requests
import pandas as pd
import names
import re
import urllib
import os
import time
import unicodedata
from datetime import datetime
start = time.time()

In [0]:
def collect_articles(base_url):
  #----------------------------------------------------
  ''' Collect all article links from HTML using base_url '''

  # connect to server, randomize user-agent tag to enable prolonged iterations
  req = requests.get(base_url, headers=({'User-Agent' : names.get_full_name()}))
  # get the html content from the webpage
  r = req.content
  # 'soup' it
  soup = bsp(r, 'lxml')

  #We only want 2019 rn 
  nba_articles = []
  for tag in soup.findAll('a', href=True):
      nba_articles.append(tag['href'])
      #print (tag['href'])
      
      

  #nba_articles[1:1000]

  #Getting new links for only articles written about NBA
  #Scraping brought up a lot of articles related to espn but not NBA 

  nba_url = [s for s in nba_articles if 'http://www.espn.com/nba/' in s]
  return(nba_url)

In [0]:
def get_titles(nba_url):
  #start = time.time()

  #os.chdir("interim/")
  df = pd.DataFrame(columns=['Body', 'Date', 'Title'])
  name = ['']
  for URL in nba_url:
      page = requests.get(URL)
      soup = bsp(page.content, 'html.parser')
      results = soup.find(id='article-feed')
      #print(results.prettify())
      if results.find_all('div', class_='container') == "NoneType":
              break
      job_elems = results.find_all('div', class_='container')
      
      for job_elem in job_elems:
          # Each job_elem is a new BeautifulSoup object.
          # You can use the same methods on it as you did before.
          title_elem = job_elem.find('header', class_='article-header')
          body_elem = job_elem.find('div', class_='article-body')
          date_elem = job_elem.find('span', class_='timestamp')
          #make into pandas dataframe to import R 
          if None in (title_elem, body_elem, date_elem):
              continue
          # {Title, Date, Body, Link}
          print("Retrieving information for: ", title_elem.text.strip())
          d = {"Title": [title_elem.text.strip()], 'Body': [body_elem.text.strip()], "Date": [date_elem.text.strip()]}
          #making df for R 
          new_df = pd.DataFrame(data = d)
          name = date_elem
      name = date_elem
      #print(name)
      df = df.append(new_df)
      return(df)

In [0]:
def get_articles():
  ''' Iteratively obtain URLs '''
  current_year = int(datetime.now().year)
  url_template = 'http://www.espn.com/nba/news/archive/_/month/{month}/year/{year}'
  article_list = []
  months_list = ['january','february','march','april','may','june','july','august','september','october','november','december']
  df = pd.DataFrame(columns=['Body', 'Date', 'Title'])

  for year in range(2003, current_year+1):

    for month in months_list:

      try:
        base_url = url_template.format(month = month,year = year)
        nba_url = collect_articles(base_url)
        df = df.append(get_titles(nba_url))
        #print(month,year)
      except:
        print(str(month) + str(year) + "doesn't work")
  
  return(df)


In [0]:
df = get_articles()

In [0]:

df.to_csv("NBA ESPN Articles.csv")